In [ ]:
"""
Insert all the feature names
"""

all_features = {'StepNeg': ['time_constant'], 'StepPos':['mean_frequency']}


%load_ext autoreload
%autoreload

from allensdk.api.queries.biophysical_api import BiophysicalApi
%matplotlib notebook
import matplotlib.pyplot as plt
import collections

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import numpy
import json

import efel



# Load the .nwb file containing the voltage traces
from allensdk.core.nwb_data_set import NwbDataSet
nwb_data = NwbDataSet('data/475049288.nwb')

# Store data in a dictionary step_name:sweep_num
steps_dict = collections.OrderedDict(sorted({'StepNeg':23,'StepPos':36}.items(), key = lambda x:x[0]))

def get_data(dataset,sweeps_dict):
    # Correct for the liquid junction potential
    junction_potential = -14
    data = collections.OrderedDict()
    for step, sweep in sweeps_dict.items():
        sweep_data = {}
        # Get sweep from NWB dataset
        sweep = dataset.get_sweep(sweep)

        # Get stimulus from the sweep
        # stimulus = sweep['stimulus']

        # Sampling rate is in Hz
        sampling_rate = sweep['sampling_rate']
        
        # Start/stop indices that exclude the experimental test pulse (if applicable)
        index_range = sweep['index_range']
        
        # Response is a numpy array in mV
        sweep_data['V'] = sweep['response'][index_range[0]:index_range[1]]*1000+junction_potential    
        
        # Define some time points in milliseconds (i.e., convert to absolute time)
        sweep_data['T'] = numpy.arange(0,(len(sweep_data['V'])/sampling_rate),1./sampling_rate)*1000
        data[step] = sweep_data
    return data

data_dict = get_data(nwb_data, steps_dict)

# Extract features
    
import efel
def get_features(data, feature_names):
    
    traces_pas = []
    traces_act = []
    for step_name, step_traces in data.items():
        trace = {}
        trace['T'] = data[step_name]['T']
        trace['V'] = data[step_name]['V']
        trace['stim_start'] = [270]
        trace['stim_end'] = [1270]
        trace['name'] = step_name
    
        if 'Pos' in step_name:
            traces_act.append(trace)       
        else:
            traces_pas.append(trace)
            
    features_values = efel.getMeanFeatureValues(traces_pas, feature_names['StepNeg'])
    features_values_act = efel.getMeanFeatureValues(traces_act, feature_names['StepPos']) 
    
    features_values.extend(features_values_act)
    return features_values

efel_features = get_features(data_dict, all_features)

# Write features mean values and std in a json file
features_dict = collections.OrderedDict()
for step_name, features in zip(steps_dict.iterkeys(), efel_features):
    features_dict[step_name] = {"soma":{}}
    for name, value in features.items():
        features_dict[step_name]["soma"][name] = [value, abs(value*0.05)]    
    
with open('./config/features.json', 'w') as fp:
    json.dump(features_dict, fp, indent = 4)


In [ ]:
# Score a student's dictionary of parameters
student_name = 'me2'

student_params = {u'ek.all': -107, u'e_pas.somatic': -63.3146, u'Ra.basal': 50.8833, 
               u'gbar_Kv3_1.axonal': 0.0100065, u'gbar_NaV.basal': 0.0205486, 
               u'gbar_Kv3_1.basal': 0.205458, u'e_pas.basal': -62.7085, 
               u'gbar_K_T.axonal': 0.004327727402343744, u'gamma_CaDynamics.somatic': 
               0.00169922, u'v_init': -81, u'cm.basal': 0.6721357746112606, 
               u'gbar_Ca_HVA.axonal': 3.83558e-05, u'gbar_NaV.somatic': 0.03811402789488886, 
               u'gbar_Kv2like.axonal': 0.0010615, u'gbar_Ca_LVA.somatic': 0.0060162, 
               u'gbar_Im_v2.basal': 0.00949979, u'gbar_SK.somatic': 0.0001051137, 
               u'g_pas.basal': 1.10433e-07, u'ena.all': 53.0, u'g_pas.axonal': 0.00200656, 
               u'gamma_CaDynamics.axonal': 0.0477399, u'gbar_SK.axonal': 0.00661877, 
               u'cm.axonal': 1.301052033032655, u'gbar_Ca_HVA.somatic': 5.9455e-05, 
               u'gbar_Kd.axonal': 0.00833495, u'gbar_Ih.basal': 0.0007, u'gbar_Ca_LVA.axonal': 
               0.00775489, u'Ra.somatic': 72.0757, u'e_pas.axonal': -71.1766, u'gbar_NaV.axonal': 
               0.08357652681547659, u'decay_CaDynamics.axonal': 811.497, u'cm.somatic': 
               7.9717441709009265, u'gbar_Ih.somatic': 3.46139e-06, u'g_pas.somatic': 7.60303e-05, 
               u'celsius': 34, u'Ra.axonal': 100.799, u'gbar_Kv3_1.somatic': 0.0021061531405053577, 
               u'decay_CaDynamics.somatic': 326.442}
 
import cell_model
import cell_evaluator

param_file = 'students_par/parameters_'+str(student_name)+'.json'
with open(param_file, 'w+') as fp:
    json.dump(student_params, fp)
    
param_configs = json.load(open(param_file))
parameters = cell_model.define_parameters()

morphology_file = 'Sst-IRES-Cre_Ai14_IVSCC_-183332.05.02.01_486041253_m.swc'
morphology = ephys.morphologies.NrnFileMorphology(morphology_file, do_replace_axon=True)

# Define mechanisms
!nrnivmodl modfiles
mechanisms = cell_model.define_mechanisms()

# Create cell model
abi_cell = ephys.models.CellModel('abi_cell', morph=morphology, mechs=mechanisms, params=parameters)

# Define protocols
proto_configs = json.load(open('config/protocols.json'))
fitness_protocols = cell_evaluator.define_protocols()

# Set-up a simulation and a fitness calculator
sim = ephys.simulators.NrnSimulator()
fitness_calculator = cell_evaluator.define_fitness_calculator(fitness_protocols)

abi_cell.unfreeze(abi_cell.params)
evaluator = ephys.evaluators.CellEvaluator(                                          
        cell_model=abi_cell,                                                       
        param_names=[],                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)  


student_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values = student_params)

scores =  fitness_calculator.calculate_scores(student_responses)
sum = numpy.sum([value for value in scores.values()])
avg = numpy.mean([value for value in scores.values()])

print "\nSum of all feature values: " + str(sum)
print "Average of the feature values: " + str(avg)

student_score = {student_name: [sum, avg]}

fd = open('final_scores.csv','a')
fd.write('\n'+str(sum)+','+str(avg)+','+student_name)
fd.close()